<h3>Applications to Finance</h3>

<p>In this final chapter, we will use pandas to perform a number of different financial
analyses of stock data obtained from Yahoo! Finance. We will briefly cover a number of
topics in financial analysis. The focus will be on using pandas to derive results from the
domain of finance, specifically, time-series stock data, and not on details of the financial
theory.</p><p>Specifically, in this chapter, we will progress through the following tasks:</p>
    <ul>
    <li>Fetching and organizing stock data from Yahoo!</li>
<li>Plotting time-series prices</li>
<li>Plotting volume-series data</li>
<li>Calculating simple daily percentage change</li>
<li>Calculating simple daily cumulative returns</li>
<li>Resampling data from daily to monthly returns</li>
<li>Analyzing distribution of returns</li>
<li>Performing a moving-average calculation</li>
<li>Comparing average daily returns across stocks</li>
<li>Correlating stocks based on the daily percentage change of closing price</li>
<li>Volatility calculation</li>
<li>Determining risk relative to expected returns</li>
</ul>

In [12]:
# necessary imports for the workbook
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt

# Set some pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 78)
pd.set_option('precision', 4)

# do all our graphics inline
%matplotlib inline

<h3>Obtaining and organizing stock data from
Yahoo!</h3>

<p>The first step we will take is to write a couple of functions that help us with retrieving
stock data from Yahoo! Finance. We have already seen that this data can be read using a
pandas DataReader object, but we will need to organize the data a little differently than
how it is provided by Yahoo! as we are going to perform various pivots of this information
later.</p><p>
To facilitate this, we will start with the following function to get all the Yahoo! data for a
specific stock between the two specified dates and also add the stock’s symbol in a column
for each entry. This will be needed later for pivots:</p>

In [15]:
# read data from Yahoo! Finance for a specific
# stock specified by ticker and between the start and end dates
def getStockData(ticker, start, end):
    # read the data
    data = web.DataReader(ticker, "yahoo", start, end)
    # rename this column
    data.rename(columns={'Adj Close': 'AdjClose'}, inplace=True)
    # insert in the ticker as a column
    data.insert(0, "Ticker", ticker)
    return data

For consistency and reproducibility by the reader, examples in this chapter will use a fixed
3-year window spanning the years of 2012 through 2014. The following reads data for that
3-year period for the MSFT ticker and prints the first 5 rows:

In [16]:
# request the three years of data for MSFT
start = datetime.datetime(2012, 1, 1)
end = datetime.datetime(2014, 12, 31)
getStockData("MSFT", start, end).head()

           Ticker   High    Low  ...  Close      Volume  AdjClose
Date                             ...                             
2012-01-03   MSFT  26.96  26.39  ...  26.77  6.4732e+07   21.9596
2012-01-04   MSFT  27.47  26.78  ...  27.40  8.0516e+07   22.4764
2012-01-05   MSFT  27.73  27.29  ...  27.68  5.6081e+07   22.7061
2012-01-06   MSFT  28.19  27.53  ...  28.11  9.9456e+07   23.0588
2012-01-09   MSFT  28.10  27.72  ...  27.74  5.9707e+07   22.7553

[5 rows x 7 columns]

Now that we have a function that can get data for a single ticker, it will be convenient to
have a function that can read the data for multiple tickers and return them all in a single
data structure. The following code performs this task:

In [18]:
# gets data for multiple stocks
# tickers: a list of stock symbols to fetch
# start and end are the start and end dates
def getDataForMultipleStocks(tickers, start, end):
    # we return a dictionary
    stocks = dict()
    # loop through all the tickers
    for ticker in tickers:
        # get the data for the specific ticker
        s = getStockData(ticker, start, end)
        # add it to the dictionary
        stocks[ticker] = s
        # return the dictionary
    return stocks

<p>The examples in this chapter will use historical quotes for Apple (AAPL), Microsoft
(MSFT), General Electric (GE), IBM (IBM), American Airlines (AA), Delta Airlines
(DAL), United Airlines (UAL), Pepsi (PEP), and Coca Cola (KO).</p><p>
These stocks were chosen deliberately to have a sample of multiple stocks in each of three
different sectors: technology, airlines, and soft drinks. The purpose of this is to
demonstrate how to derive correlations in various stock price measurements over the
selected time period among the stocks in similar sectors and to also demonstrate the
difference in stocks between sectors.</p><p>
We can read all of these with the following code, which uses the function we just created:</p>

In [19]:
# get the data for all the stocks that we want
raw = getDataForMultipleStocks(["MSFT", "AAPL", "GE", "IBM", "AA", "DAL",
"UAL", "PEP", "KO"], start, end)

In [20]:
# take a peek at the data for MSFT
raw['MSFT'][:5]

           Ticker   High    Low  ...  Close      Volume  AdjClose
Date                             ...                             
2012-01-03   MSFT  26.96  26.39  ...  26.77  6.4732e+07   21.9596
2012-01-04   MSFT  27.47  26.78  ...  27.40  8.0516e+07   22.4764
2012-01-05   MSFT  27.73  27.29  ...  27.68  5.6081e+07   22.7061
2012-01-06   MSFT  28.19  27.53  ...  28.11  9.9456e+07   23.0588
2012-01-09   MSFT  28.10  27.72  ...  27.74  5.9707e+07   22.7553

[5 rows x 7 columns]

We will be particularly interested in the adjusted close values in the AdjClose column.
However, it would be most convenient for us if we had a DataFrame object indexed by
date but where each column is the stock ticker for one stock and the values in each column
are the adjusted close value for that ticker at that date. We can have this format if we pivot
the data around. The following function will do this for us: